# Mathematical Model: Dual Fourier Series Resonator Model

**Focus:** Theory section - Mathematical model and related components

This notebook focuses on the mathematical model used to replicate traditional sasando acoustic characteristics in electric versions.


## Notation

### Time Domain Variables
- $t$: Time (seconds)
- $f(t)$: Signal as a function of time
- $E(t)$: Amplitude envelope function
- $A_0$: Initial amplitude of the envelope
- $\tau$: Decay time constant (seconds)

### Frequency Domain Variables
- $f_0$: Fundamental frequency (Hz)
- $\omega_0 = 2\pi f_0$: Fundamental angular frequency (rad/s)
- $n$: Harmonic index (integer: 0, 1, 2, ...)
- $nf_0$: Frequency of the $n$-th harmonic (Hz)
- $M_n$: Magnitude of the $n$-th harmonic from FFT

### Fourier Series Coefficients
- $A_n$: Amplitude coefficient for the $n$-th harmonic
- $\phi_n$: Phase coefficient for the $n$-th harmonic
- $\alpha_n$: Frequency-dependent amplitude scaling factor for the $n$-th harmonic
- $\theta$: Phase shift introduced by the resonator (radians)

### Signal Processing
- $F_s$: Sampling rate (Hz) - typically 48,000 Hz
- $N$: Number of samples
- $T$: Signal duration (seconds)
- $\text{FFT}[\cdot]$: Fast Fourier Transform operator

### Model Parameters
- $\text{num\_harmonics}$: Number of harmonics used in the model (400 in this work)
- $\text{SASANDO\_FREQ\_MIN} = 98$ Hz: Minimum frequency of sasando resonator range
- $\text{SASANDO\_FREQ\_MAX} = 1047$ Hz: Maximum frequency of sasando resonator range


## Assumptions

The mathematical model is based on the following assumptions:

1. **Harmonic Structure**: The signal can be accurately represented as a sum of harmonics (integer multiples of the fundamental frequency). This assumes that inharmonic partials are negligible or can be approximated by nearby harmonics.

2. **Linear Resonator Model**: The resonator's effect is modeled as a linear transformation with frequency-dependent amplitude scaling $\alpha_n$ and constant phase shift $\theta$. This assumes the resonator behaves linearly and does not introduce significant non-linear distortion.

3. **Exponential Decay**: The amplitude envelope follows an exponential decay model $A_0 e^{-t/\tau}$. This assumes that the decay is uniform across all harmonics and that the decay time constant $\tau$ is constant throughout the signal.

4. **Stationary Signal**: The signal characteristics (fundamental frequency, harmonic amplitudes, phases) are assumed to be stationary over the analysis window, except for the exponential decay envelope.

5. **Perfect Harmonic Alignment**: The model assumes that all frequency components are exact integer multiples of the fundamental frequency $f_0$. In reality, slight frequency variations may occur.

6. **Noise-Free Extraction**: The feature extraction process assumes that noise and continuous spectral content do not significantly affect the extraction of harmonic amplitudes and phases.

7. **Resonator Frequency Response**: The resonator's frequency response is approximated by a step function:
   $$
   \alpha_n = \begin{cases}
       0.8 & \text{if } 98 \text{ Hz} \leq n \cdot f_0 \leq 1047 \text{ Hz} \\
       0.2 & \text{otherwise}
   \end{cases}
   $$
   This is a simplification of the actual complex frequency response.

8. **Constant Phase Shift**: The phase shift $\theta$ introduced by the resonator is assumed to be constant across all frequencies. In reality, phase shifts may vary with frequency.


## Mathematical Model

To model the interaction between the electric sasando signal and the resonator, we use a dual Fourier series representation:

$$
f(t) = \sum_{n=0}^{\infty} A_n \cos\left(n\omega_0 t + \phi_n\right) + \sum_{n=0}^{\infty} \alpha_n A_n \cos\left(n\omega_0 t + \phi_n + \theta\right)
$$

Where:
- $\omega_0 = 2\pi f_0$ is the fundamental frequency in rad/s
- $A_n$ are the amplitude coefficients for each harmonic (extracted from the signal)
- $\phi_n$ are the phase coefficients for each harmonic (extracted from the signal)
- $\alpha_n$ is the frequency-dependent amplitude scaling factor for the $n$-th harmonic (models resonator response)
- $\theta$ is the phase shift introduced by the resonator (constant, typically $\pi/4$ rad)

The first sum represents the original signal decomposed into harmonics, while the second sum represents the resonator's contribution with frequency-dependent scaling $\alpha_n$ and phase shift $\theta$.

### Model Components

**First Series:** $\sum_{n=0}^{\infty} A_n \cos(n\omega_0 t + \phi_n)$
- Represents the original signal decomposed into harmonics
- $A_n$: Amplitude coefficient for the $n$-th harmonic (extracted from input signal)
- $\phi_n$: Phase coefficient for the $n$-th harmonic (extracted from input signal)
- $\omega_0 = 2\pi f_0$: Fundamental frequency in rad/s
- $n$: Harmonic index (0 = DC, 1 = fundamental, 2, 3, ... = harmonics)

**Second Series:** $\sum_{n=0}^{\infty} \alpha_n A_n \cos(n\omega_0 t + \phi_n + \theta)$
- Represents the resonator's contribution to the sound
- $\alpha_n$: Frequency-dependent amplitude scaling factor (models resonator response)
  - Larger values (0.8) for frequencies in sasando range (98-1047 Hz)
  - Smaller values (0.2) for frequencies outside this range
- $\theta$: Phase shift introduced by the resonator (constant, typically $\pi/4$ rad)


## Resonator Response Function

The function $\alpha_n$ models the frequency-dependent amplification of the sasando resonator:

$$
\alpha_n = \begin{cases}
    0.8 & \text{if } 98 \text{ Hz} \leq n \cdot f_0 \leq 1047 \text{ Hz} \\
    0.2 & \text{otherwise}
\end{cases}
$$

This reflects the resonator's natural amplification of frequencies within the sasando's characteristic range (98-1047 Hz). The resonator naturally amplifies frequencies in this range more strongly (0.8) compared to frequencies outside this range (0.2).


## Amplitude Envelope and Decay

Plucked string instruments exhibit exponential decay in amplitude over time. To model this, we multiply the steady-state model by an exponential envelope:

$$
f_{\text{decay}}(t) = E(t) \cdot f(t) = A_0 e^{-t/\tau} \cdot \left[ \sum_{n=0}^{\infty} A_n \cos\left(n\omega_0 t + \phi_n\right) + \sum_{n=0}^{\infty} \alpha_n A_n \cos\left(n\omega_0 t + \phi_n + \theta\right) \right]
$$

Where:
- $A_0$ is the initial amplitude (extracted from the signal's peak envelope)
- $\tau$ is the decay time constant (fitted from the signal's envelope using exponential regression)

The envelope is extracted using the Hilbert transform, which provides an accurate representation of the signal's amplitude modulation.

### Decay Model

The exponential decay model captures the natural decay characteristics of plucked strings:

$$
E(t) = A_0 e^{-t/\tau}
$$

Where:
- $A_0$: Initial amplitude at $t=0$
- $\tau$: Decay time constant (time for amplitude to decay to $1/e \approx 0.368$ of initial value)
- Smaller $\tau$ values indicate faster decay
- Larger $\tau$ values indicate slower decay


## A$_n$ Extraction Formula

When extracting the amplitude coefficients $A_n$ from a recorded signal (which already includes resonator effects), we must account for the resonator's contribution. The FFT magnitude $M_n$ at frequency $n\omega_0$ represents the combined magnitude of both series terms.

### Derivation

Starting with the sum of two cosine terms:
$$
s(t) = A_n \cos(n\omega_0 t + \phi_n) + \alpha_n A_n \cos(n\omega_0 t + \phi_n + \theta)
$$

Using the trigonometric identity $\cos(a + b) = \cos(a)\cos(b) - \sin(a)\sin(b)$:

$$
s(t) = A_n \cos(n\omega_0 t + \phi_n) + \alpha_n A_n [\cos(n\omega_0 t + \phi_n)\cos(\theta) - \sin(n\omega_0 t + \phi_n)\sin(\theta)]
$$

$$
= A_n [1 + \alpha_n \cos(\theta)] \cos(n\omega_0 t + \phi_n) - \alpha_n A_n \sin(\theta) \sin(n\omega_0 t + \phi_n)
$$

This can be written as a single sinusoid $R \cos(n\omega_0 t + \phi_n + \delta)$, where the magnitude $R$ is:

$$
R = A_n \sqrt{[1 + \alpha_n \cos(\theta)]^2 + [\alpha_n \sin(\theta)]^2}
$$

$$
= A_n \sqrt{1 + 2\alpha_n \cos(\theta) + \alpha_n^2 \cos^2(\theta) + \alpha_n^2 \sin^2(\theta)}
$$

$$
= A_n \sqrt{1 + 2\alpha_n \cos(\theta) + \alpha_n^2(\cos^2(\theta) + \sin^2(\theta))}
$$

$$
= A_n \sqrt{1 + 2\alpha_n \cos(\theta) + \alpha_n^2}
$$

Since $M_n$ (from FFT) represents the magnitude $R$ of the combined signal, we solve for $A_n$:

$$
A_n = \frac{M_n}{\sqrt{1 + 2\alpha_n \cos(\theta) + \alpha_n^2}}
$$

This formula correctly accounts for the phase relationship between the two cosine terms when extracting the base amplitude coefficient.

### Physical Interpretation

- $M_n$: The measured magnitude from FFT (includes both direct signal and resonator contribution)
- $A_n$: The base amplitude before resonator effect is applied
- The denominator $\sqrt{1 + 2\alpha_n \cos(\theta) + \alpha_n^2}$ accounts for how the resonator amplifies and phase-shifts the signal
- When $\alpha_n = 0$ (no resonator), $A_n = M_n$ (as expected)
- When $\theta = 0$ (no phase shift), the formula simplifies to $A_n = M_n / (1 + \alpha_n)$


## Complete Model with Decay

Combining all components, the complete model is:

$$
f_{\text{complete}}(t) = A_0 e^{-t/\tau} \cdot \left[ \sum_{n=0}^{N} A_n \cos\left(n\omega_0 t + \phi_n\right) + \sum_{n=0}^{N} \alpha_n A_n \cos\left(n\omega_0 t + \phi_n + \theta\right) \right]
$$

Where:
- $N = 399$ (400 harmonics: $n = 0$ to $399$)
- $A_0$: Initial amplitude (from signal envelope peak)
- $\tau$: Decay time constant (fitted from signal envelope)
- $A_n$: Extracted using the formula above
- $\phi_n$: Extracted from FFT phase spectrum
- $\alpha_n$: Determined by resonator response function
- $\theta = \pi/4$ radians (45 degrees)

### Model Summary

The model consists of three main components:

1. **Harmonic Decomposition**: Signal represented as sum of harmonics with amplitudes $A_n$ and phases $\phi_n$
2. **Resonator Effect**: Second series adds frequency-dependent amplification ($\alpha_n$) and phase shift ($\theta$)
3. **Temporal Decay**: Exponential envelope ($A_0 e^{-t/\tau}$) captures natural decay of plucked strings


## References

- Oppenheim, A. V., & Schafer, R. W. (2010). *Discrete-Time Signal Processing* (3rd ed.). Prentice Hall. Section 2.6: Sinusoidal Signals and Phasors.
- Proakis, J. G., & Manolakis, D. G. (2007). *Digital Signal Processing* (4th ed.). Prentice Hall. Chapter 2: Discrete-Time Signals and Systems.
